# Exercices Seance 06

## 1/ XML to CSV

Écrire une fonction qui lit les 5 fichiers XML de Balzac, créé et sauvegarde un dataframe avec 8 colonnes : titre, auteur, gender, genre, canon, date_publication, date_naissance, date_mort


In [1]:
import os
from lxml import etree
import pandas as pd
from tqdm.notebook import tqdm
from glob import glob

In [3]:
path_to_folder_xml = "/home/crazyjeannot/Documents/cours/2024/Python4DH/data/xml/*.xml"

In [4]:
def trouve_info(path_to_xml):
    tree = etree.parse(path_to_xml)

    titre = tree.find(".//title").text
    auteur = tree.find(".//author").get('name')
    gender = tree.find(".//author").get('sex')
    date_naissance = tree.find(".//author").get('from')
    date_mort = tree.find(".//author").get('to')
    date_publication = tree.findall(".//date")[1].get('when')
    canon = tree.find('.//profileDesc').get('tag')
    genre = ' '.join([element.text for element in tree.findall('.//term')]) 

    return titre, auteur, gender, genre, canon, date_publication, date_naissance, date_mort


In [5]:
def moulinette(path_to_folder_xml):
    df_corpus = pd.DataFrame([], columns=['titre', "auteur", "gender", "genre", "canon", "date_publication", "date_naissance", "date_mort"])
    for doc in tqdm(glob(path_to_folder_xml)):
        titre, auteur, gender, genre, canon, date_publication, date_naissance, date_mort = trouve_info(doc) 
    
        df_roman = pd.DataFrame([(titre, auteur, gender, genre, canon, date_publication, date_naissance, date_mort)], columns=['titre', "auteur", "gender", "genre", "canon", "date_publication", "date_naissance", "date_mort"])
        df_corpus = pd.concat([df_corpus, df_roman])
    return df_corpus

In [6]:
df = moulinette(path_to_folder_xml)

  0%|          | 0/5 [00:00<?, ?it/s]

In [7]:
df

,titre,auteur,gender,genre,canon,date_publication,date_naissance,date_mort
0,Le Chef-d'œuvre inconnu,"Balzac, Honoré de",male,cycles et séries nouvelles,canon,1845,1799,1850
0,Eugénie Grandet,"Balzac, Honoré de",male,cycles et séries,canon,1843,1799,1850
0,Le Père Goriot,"Balzac, Honoré de",male,cycles et séries,canon,1843,1799,1850
0,Sarrasine,"Balzac, Honoré de",male,cycles et séries nouvelles,canon,1844,1799,1850
0,Illusions perdues,"Balzac, Honoré de",male,cycles et séries,canon,1843,1799,1850


## 2/ XML to txt 

Écrire une fonction qui convertie les 5 fichiers XML en fichiers txt.

In [46]:
path_to_folder_txt = "/home/crazyjeannot/Documents/cours/Python4DH/data/txt/"

In [51]:
def xml_to_txt(path_xml, path_name_res):
    
    chaine = ""
    doc_name = os.path.splitext(os.path.basename(path_xml))[0]
        
    tree = etree.parse(path_xml)

    if tree.findall(".//p"):
        for line in tree.findall(".//p"):
            if line.text:
                chaine += line.text
    else:
        print(doc_name)
                    
    with open(path_name_res+doc_name+".txt", "w", encoding="utf8") as file_out:
        file_out.write(chaine)

In [52]:
def moulinette_xmltotxt(path_name, path_name_res):
    for doc in tqdm(glob(path_name)):
        xml_to_txt(doc, path_name_res)

In [53]:
moulinette_xmltotxt(path_to_folder_xml, path_to_folder_txt)

  0%|          | 0/5 [00:00<?, ?it/s]

## 3/ Exploiter des JSON

À partir des .book (json customisés) envoyés avec ce notebook, écrire une fonction qui ouvre les documents, une autre qui récupère la liste des verbes ('agent', 'patient') et des adjectifs ('mod') associés à ces personnages (les 10 premiers), une autre qui calcule leur fréquence d'apparition relative au nombre total de verbes / adjectifs et retourne le résultat sour la forme d'un dataframe (colonnes = mots, ligne = txt_j_personnage_i, valeurs = fréquence relative).

Enfin définir une fonction main qui appelle ces différentes fonctions, fusionne les dataframes au fur et à mesure puis retourne le résultat sous la forme d'un dataframe (colonnes = mots, ligne = txt_j_personnage_i, valeurs = fréquence relative). 

Sauver ce dataframe final. Me l'envoyer avec votre notebook. 

Vous pouvez utiliser la fonction get_characterization vue dans le cours précédent.

Vous pouvez également utiliser les fonctions déjà définies dans les différents exercices corrigés des semaines précédentes

In [60]:
import json
from collections import Counter

In [61]:
path_book = "/home/crazyjeannot/Documents/cours/Python4DH/data/booknlp_output/booknlp_output_book/1830_Stendhal_Le-Rouge-et-le-noir.book"

In [62]:
path_books = "/home/crazyjeannot/Documents/cours/Python4DH/data/booknlp_output/booknlp_output_book/*.book"

In [63]:
def read_booknlp(path_book):
    with open(path_book, "r") as booknlp_book:
        book_data = json.load(booknlp_book)
    return book_data

In [64]:
def get_characterization(booknlp_data, N):
    list_verb_suj, list_verb_obj, list_adj = [], [], []
    for i in range(N):
        list_verb_suj.append([item['w'] for item in booknlp_data["characters"][i]['agent']])
        list_verb_obj.append([item['w'] for item in booknlp_data["characters"][i]['patient']])
        list_adj.append([item['w'] for item in booknlp_data["characters"][i]['mod']])
    return list_verb_suj, list_verb_obj, list_adj

In [65]:
def get_frequence(list1, list2, list3):
    list_dict_freq = []
    for list_vsuj, list_vobj, list_adj in zip(list1, list2, list3):
        conteur_mots_booknlp = dict(Counter(list_vsuj+list_vobj+list_adj))
        dict_freq = {}
        for key, value in conteur_mots_booknlp.items():
            dict_freq[key] = conteur_mots_booknlp[key]/sum(conteur_mots_booknlp.values())
        list_dict_freq.append(dict_freq) 
    return list_dict_freq

In [66]:
def main(path_files, N=10):
    df_main = pd.DataFrame()
    index = []
    i=1
    for doc in tqdm(glob(path_files)):
        data_booknlp = read_booknlp(doc)
        list_verb_suj, list_verb_obj, list_adj = get_characterization(data_booknlp, N)
        list_dict_freq = get_frequence(list_verb_suj, list_verb_obj, list_adj)
        df_temp = pd.DataFrame(list_dict_freq)
        df_main = pd.concat([df_main, df_temp])
        index.extend(["txt_"+str(i)+"_personnage_"+str(j) for j in range(1, len(list_dict_freq)+1)])#txt_j_personnage_i
        i+=1
    df_main['index']=index
    return df_main.set_index("index")

In [67]:
df = main(path_books)

  0%|          | 0/7 [00:00<?, ?it/s]

/tmp/ipykernel_7547/1961434860.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_main['index']=index


In [68]:
df

,née,interrompait,parlait,plaisait,remplaçait,enfermait,atteignait,franchissait,avait,avançait,...,remua,rencontrèrent,cingla,posait,mettons,aperçois,souffleté,couchiez,Voyez,serrai
index,,,,,,,,,,,,,,,,,,,,,
txt_1_personnage_1,0.006369,0.006369,0.012739,0.006369,0.006369,0.006369,0.006369,0.006369,0.006369,0.006369,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
txt_1_personnage_2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
txt_1_personnage_3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
txt_1_personnage_4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
txt_1_personnage_5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.012821,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
txt_7_personnage_6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
txt_7_personnage_7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
txt_7_personnage_8,NaN,NaN,0.018182,NaN,NaN,NaN,NaN,NaN,0.018182,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
df.to_csv("resultat_ex_seance_06.csv")

In [51]:
import numpy as np

In [71]:
# plus simple: On prend la liste de verbes/adjectifs par roman et non par personnage ! bonus pour la suite, pas demandé dans l'exercice

In [41]:
def read_booknlp(path_book):
    with open(path_book, "r") as booknlp_book:
        book_data = json.load(booknlp_book)
    return book_data

In [42]:
def get_characterization(booknlp_data, N):
    list_verb_suj, list_verb_obj, list_adj = [], [], []
    for i in range(N):
        list_verb_suj.extend([item['w'] for item in booknlp_data["characters"][i]['agent']])
        list_verb_obj.extend([item['w'] for item in booknlp_data["characters"][i]['patient']])
        list_adj.extend([item['w'] for item in booknlp_data["characters"][i]['mod']])
    return list_verb_suj, list_verb_obj, list_adj

In [43]:
def get_frequence(list1, list2, list3, M):
    conteur_mots_booknlp = dict(Counter(list1+list2+list3).most_common(M))
    dict_freq = {}
    for key, value in conteur_mots_booknlp.items():
        dict_freq[key] = conteur_mots_booknlp[key]/sum(conteur_mots_booknlp.values())
    return dict_freq

In [53]:
def main(path_files, N=10, M=5000):
    df_main = pd.DataFrame()
    index = []
    i=1
    for doc in tqdm(glob(path_files)):
        doc_name = os.path.splitext(os.path.basename(doc))[0]

        data_booknlp = read_booknlp(doc)
        list_verb_suj, list_verb_obj, list_adj = get_characterization(data_booknlp, N)
        dict_freq = get_frequence(list_verb_suj, list_verb_obj, list_adj, M)
        df_temp = pd.DataFrame.from_dict(dict_freq, orient='index')
        df_main = pd.concat([df_main, df_temp.T])
        index.append(doc_name)
        i+=1
    df_main['index']=index
    return df_main.set_index("index")

In [54]:
df = main(path_books, N=10)

  0%|          | 0/7 [00:00<?, ?it/s]

/tmp/ipykernel_7547/2440555551.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_main['index']=index


In [55]:
df

,disait,sais,dit,ai,vois,avait,connu,vue,crois,vu,...,serrai,gros,pacifique,bienveillant,magnanime,prisonnier,face,grossier,Monsieur,patron
index,,,,,,,,,,,,,,,,,,,,,
1930_Colette_Sido,0.018895,0.015988,0.014535,0.011628,0.010174,0.008721,0.008721,0.008721,0.008721,0.008721,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1907_Leblanc-Maurice_Arsene-Lupin-gentleman-cambrioleur,0.001227,0.003067,0.018405,0.009202,0.003067,0.009202,0.003681,NaN,0.002454,0.009202,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1841_Sand-George_Un-hiver-a-Majorque,NaN,0.010076,0.012594,0.007557,0.005038,0.002519,0.002519,NaN,0.012594,0.010076,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1892_Verne-Jules_Le-Chateau-des-Carpathes,0.001967,0.003934,0.012459,0.001311,0.000656,0.013770,NaN,0.003279,0.001311,0.004590,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1830_Stendhal_Le-Rouge-et-le-noir,0.008912,0.003055,0.022406,0.009930,0.003692,0.021388,0.001400,0.000382,0.002164,0.003947,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1909_Zevaco-Michel_Nostradamus,0.004381,0.005958,0.035921,0.006483,0.002979,0.004381,0.001227,0.001227,0.002453,0.007885,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1883_Guy-de-Maupassant_Contes-de-la-Becasse,0.007692,0.003846,0.007692,0.006410,NaN,0.006410,0.001282,0.002564,0.003846,0.002564,...,0.001282,0.001282,0.001282,0.001282,0.001282,0.001282,0.001282,0.001282,0.001282,0.001282


In [56]:
df.to_csv('resultat_roman_ex_seance_06.csv', index=True)

In [58]:
df.columns[4500:]

Index(['embrassais', 'violenter', 'pressant', 'posait', 'Voyez', 'serrai',
       'gros', 'pacifique', 'bienveillant', 'magnanime', 'prisonnier', 'face',
       'grossier', 'Monsieur', 'patron'],
      dtype='object')